# Distance de Levenshtein

**Objectifs :**  
- Définir une notion de distance distance mesurant à quel point deux mots sont similaires
- Étant donné un mot et un lexique, déterminer quels mots du lexique sont les plus proches du mot fourni

**Opérations :**  
- *Remplacement* d'une lettre par une autre
- *Insertion* d'une lettre
- *Suppression* d'une lettre

**Définition (distance de Levenshtein) :**  
La distance $d(u, v)$ entre deux mots $u$ et $v$ est le plus petit entier $k$ tel que $v$ peut être obtenu à partir de $v$ par une série de $k$ insertions, remplacements ou suppressions.

**Exemples :**
- $d(\text{chateau}, \text{bateau}) = 2$  
- $d(\text{chat}, \text{batte}) = 4$  

## Calcul de la distance entre deux mots

**Observations :**
- Si les deux mots dont on cherche la distance commencent par la même lettre, il n'est jamais avantageux de faire une insertion ou une suppression sur cette lettre (ni évidemment un remplacement).  
- Si les deux mots commencent par des lettres différentes, on ne peut pas savoir quelle opération il est préférable d'appliquer sur ce couple de lettres sans regarder la suite (exemples : chat et hat, chat et achat, plat et flat).

**Définition inductive :**  
- La distance entre un mot vide et un autre mot est la longueur de cet autre mot.
- La distance entre deux mots non vides est obtenue en considérant le calcul suivant : 
  $$
  d(xu, yv) = 
  \begin{cases}
  1 + \min \left\{ d(u, yv), d(u, v), d(xu, v) \right\} & \text{ si } x \neq y \\
  d(u, v) & \text{ sinon}
  \end{cases}
  $$

### Implémentation récursive

In [136]:
def distance_rec(un_mot, autre_mot):
    if un_mot == "":
        return len(autre_mot)
    elif autre_mot == "":
        return len(un_mot)
    else:
        d_rep = distance_rec(un_mot[1:], autre_mot[1:])
        if un_mot[0] == autre_mot[0]:
            return d_rep
        else:
            d_ins = distance_rec(un_mot[1:], autre_mot)
            d_del = distance_rec(un_mot, autre_mot[1:])
            return 1 + min(d_rep, d_ins, d_del)


In [137]:
distance_rec("", "chat")

4

In [138]:
distance_rec("chat", "")

4

In [139]:
distance_rec("chateau", "bateau")

2

In [140]:
distance_rec("chat", "batte")

4

In [141]:
distance_rec("anticonstitutionnellement", "c'est n'importe quoi")

KeyboardInterrupt: 

### Implémentation itérative

Une autre approche consiste à construire ligne par ligne un tableau indiquant la distance entre chaque préfixe des deux mots. Une nouvelle case peut être remplie en regardant le contenu des trois cases en haut, à gauche et en diagonale (vers le haut et la gauche).

Ce type d'approche qui consiste à construire un résultat en s'appuyant sur un tableau de résultats à des sous-problèmes est souvent appelé *programmation dynamique*.

**Exemple :**

|       | **-** | **c** | **h** | **a** | **t** | **e** | **a** | **u** |
|-------|-------|-------|-------|-------|-------|-------|-------|-------|
| **-** |   0   |   1   |   2   |   3   |   4   |   5   |   6   |   7   |
| **b** |   1   |   ?   |       |       |       |       |       |       |
| **a** |   2   |       |       |       |       |       |       |       |
| **t** |   3   |       |       |       |       |       |       |       |
| **e** |   4   |       |       |       |       |       |       |       |
| **a** |   5   |       |       |       |       |       |       |       |
| **u** |   6   |       |       |       |       |       |       |       |


In [142]:
def distance_iter(un_mot, autre_mot):
    m, n = len(un_mot), len(autre_mot)
    i, j = 0, 0
    tableau = [list(range(n + 1))]
    for i in range(1, m+1):
        ligne = [i]
        tableau.append(ligne)
        for j in range(1, n+1):
            if un_mot[i-1] == autre_mot[j-1]:
                ligne.append(tableau[i-1][j-1])
            else:
                d_ins = tableau[i][j-1]
                d_del = tableau[i-1][j]
                d_rep = tableau[i-1][j-1]
                d = 1 + min(d_ins, d_rep, d_del)
                ligne.append(d)
    return tableau[-1][-1]

In [143]:
distance_iter("", "chat")

4

In [144]:
distance_iter("chat", "")

4

In [145]:
distance_iter("chateau", "bateau")

2

In [146]:
distance_iter("chat", "batte")

4

In [147]:
distance_iter("anticonstitutionnellement", "c'est n'importe quoi")

21

On peut améliorer un peu la consommation de mémoire de cette fonction en ne mémorisant que la ligne en cours et la ligne précédente du tableau :

In [154]:
def distance_iter_opti(un_mot, autre_mot):
    m, n = len(un_mot), len(autre_mot)
    i, j = 0, 0
    prec = list(range(n + 1))
    ligne = [0] * (n + 1)
    for i in range(1, m+1):
        ligne[0] = i
        for j in range(1, n+1):
            if un_mot[i-1] == autre_mot[j-1]:
                ligne[j] = prec[j-1]
            else:
                d_ins = ligne[j-1]
                d_del = prec[j]
                d_rep = prec[j-1]
                d = 1 + min(d_ins, d_rep, d_del)
                ligne[j] = d
        prec, ligne = ligne, prec
    return prec[-1]

In [155]:
distance_iter_opti("", "chat")

4

In [156]:
distance_iter_opti("chat", "")

4

In [157]:
distance_iter_opti("chateau", "bateau")

2

In [158]:
distance_iter_opti("chat", "batte")

4

In [159]:
distance_iter_opti("anticonstitutionnellement", "c'est n'importe quoi")

21

### Implémentation récursive avec mémoïsation

La version récursive est plus simple, mais elle est beaucoup plus lente... Pour la rendre plus efficace on peut utiliser une astuce appelée "mémoisation" : on conserve dans un dictionnaire tous les résultats déjà connus, et avant de faire le moindre calcul, on vérifie dans le dictionnaire si la distance qu'on cherche n'a pas déjà été calculée.

In [190]:
def distance_rec_opti(un_mot, autre_mot, memo=None):
    if memo is None:
        memo = {}
    if un_mot <= autre_mot:
        instance = (un_mot, autre_mot)
    else:
        instance = (autre_mot, un_mot)
    if instance in memo:
        return memo[instance]
    if un_mot == "":
        res = len(autre_mot)
    elif autre_mot == "":
        res = len(un_mot)
    else:
        d_rep = distance_rec_opti(un_mot[1:], autre_mot[1:], memo)
        if un_mot[0] == autre_mot[0]:
            res = d_rep
        else:
            d_ins = distance_rec_opti(un_mot[1:], autre_mot, memo)
            d_del = distance_rec_opti(un_mot, autre_mot[1:], memo)
            res = 1 + min(d_rep, d_ins, d_del)
    memo[instance] = res
    return res

In [191]:
distance_rec_opti("", "chat")

4

In [192]:
distance_rec_opti("chat", "")

4

In [193]:
distance_rec_opti("chateau", "bateau")

2

In [194]:
distance_rec_opti("chat", "batte")

4

In [195]:
distance_rec_opti("anticonstitutionnellement", "c'est n'importe quoi")

21

### Une petite comparaison

In [196]:
%timeit distance_iter("bougeoir", "bavoir")
%timeit distance_iter_opti("bougeoir", "bavoir")
%timeit distance_rec("bougeoir", "bavoir")
%timeit distance_rec_opti("bougeoir", "bavoir")

31.4 µs ± 1.11 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
30.7 µs ± 3.99 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
1.38 ms ± 14.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
62.5 µs ± 731 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Quel intérêt à utiliser malgré tout la version mémoïsée (si on ne s'inquiète pas trop de la mémoire) ?

In [197]:
%%timeit
distance_rec_opti("recevoir", "percevra")
distance_rec_opti("décevoir", "percevra")
distance_rec_opti("tellement", "feulement")
distance_rec_opti("feulement", "tellement")

562 µs ± 79.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [198]:
%%timeit
memo = {}
distance_rec_opti("recevoir", "percevra", memo)
distance_rec_opti("décevoir", "percevra", memo)
distance_rec_opti("tellement", "feulement", memo)
distance_rec_opti("feulement", "tellement", memo)

291 µs ± 6.85 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [199]:
print(memo)

{('', ''): 0, ('', 'a'): 1, ('r', ''): 1, ('r', 'a'): 1, ('r', 'ra'): 1, ('ir', ''): 2, ('ir', 'a'): 2, ('ir', 'ra'): 2, ('', 'ra'): 2, ('', 'vra'): 3, ('r', 'vra'): 2, ('ir', 'vra'): 2, ('oir', ''): 3, ('oir', 'a'): 3, ('oir', 'ra'): 3, ('oir', 'vra'): 3, ('', 'evra'): 4, ('r', 'evra'): 3, ('ir', 'evra'): 3, ('oir', 'evra'): 3, ('voir', 'vra'): 3, ('voir', 'evra'): 4, ('', 'cevra'): 5, ('r', 'cevra'): 4, ('ir', 'cevra'): 4, ('oir', 'cevra'): 4, ('voir', 'cevra'): 4, ('evoir', 'evra'): 3, ('evoir', 'cevra'): 4, ('r', 'rcevra'): 5, ('ir', 'rcevra'): 5, ('oir', 'rcevra'): 5, ('voir', 'rcevra'): 5, ('evoir', 'rcevra'): 5, ('cevoir', 'cevra'): 3, ('cevoir', 'rcevra'): 4, ('ecevoir', 'ercevra'): 4, ('evoir', 'ercevra'): 5, ('cevoir', 'ercevra'): 5, ('', 'rcevra'): 6, ('', 'ercevra'): 7, ('r', 'ercevra'): 6, ('ir', 'ercevra'): 6, ('oir', 'ercevra'): 6, ('voir', 'ercevra'): 6, ('', 'percevra'): 8, ('r', 'percevra'): 7, ('ir', 'percevra'): 7, ('oir', 'percevra'): 7, ('voir', 'percevra'): 7, ('

## Correction othographique

Étant donné un mot `mot` et une liste de mots `lexique`, il suffit de chercher tous les mots à une distance minimale de `mot` dans `lexique` pour obtenir une liste de candidats à la correction (et éventuellement le mot lui même s'il apparaît dans le lexique !).

**Variantes possibles :** renvoyer tous les mots à une distance inférieure à un certain seuil, ou seulement les mots de distance minimale (s'ils sont en-dessous du seuil).

In [118]:
def corrections(mot, lexique, seuil=float("+inf"), min_seulement=True):
    d_min = float("+inf")
    candidats = []
    for candidat in lexique:
        d = distance_iter(mot, candidat)
        if min_seulement and d < d_min:
            candidats = []
            d_min = d
        if d <= seuil:
            if d == d_min or not min_seulement:
                candidats.append(candidat)
    return candidats

In [133]:
for i in range(6):
    print(i, corrections("blancha", ["balance", "balança", "blanc", "calancha", "calancherai", "balancerai"], i, False))

0 []
1 []
2 ['blanc', 'calancha']
3 ['balance', 'balança', 'blanc', 'calancha']
4 ['balance', 'balança', 'blanc', 'calancha', 'balancerai']
5 ['balance', 'balança', 'blanc', 'calancha', 'calancherai', 'balancerai']


## Pour aller plus loin

- Une structure de données (potentiellement) plus efficace : les [arbres BK](https://en.wikipedia.org/wiki/BK-tree).
- Une distance qui prend en compte les erreurs dans l'ordre des lettres : la distance de [Damerau-Levenshtein](https://fr.wikipedia.org/wiki/Distance_de_Damerau-Levenshtein).
- Comment tenir compte des informations du corpus pour améliorer la suggestion de correction ?
- Comment tenir compte de la notion de seuil pour optimiser les calculs de distance ?